# Introduction LangChain

LangChain est un framework populaire qui permet aux utilisateurs de construire rapidement des applications et des pipelines autour de grands modèles de langage. Il peut être utilisé pour les chatbots, la réponse générative aux questions, le résumé, et bien plus encore.

L'idée centrale de la bibliothèque est que nous pouvons "enchaîner" différents composants pour créer des cas d'utilisation plus avancés autour des LLM. Les chaînes peuvent être constituées de plusieurs composants provenant de plusieurs modules :

- **Modèles d'invite** : Les modèles d'invites sont, en fait, des modèles pour différents types d'invites. Comme les modèles de style "chatbot", les réponses aux questions ELI5, etc.

- **LLM** : Grands modèles de langage comme GPT-3, BLOOM, etc.

- **Agents** : Les agents utilisent les LLM pour décider des actions à entreprendre, des outils tels que la recherche sur le web ou des calculatrices peuvent être utilisés, le tout étant intégré dans une boucle logique d'opérations.

- **Mémoire** : Mémoire à court terme, mémoire à long terme.

Mais commençons simplement avec les élements de base.

In [1]:
# get the environments var
import os
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv())

# get hugging face API key
HUGGINGFACEHUB_API_TOKEN = os.environ["HUGGINGFACEHUB_API_TOKEN"]


In [9]:
# to authenticate in notebook (from jupyter or colab only)
#from huggingface_hub import notebook_login
#notebook_login()

## Using simple base model FLAN-T5 (Google)

Une chaîne simple est la réunion d'un LLM et d'un template de prompt (Prompt template). Nous allons commencer par montrer qu'il est possible de créer une chaîne simple avec un "petit" LLM comme `FLAN-T5` de Google.

On va se servir de la plateforme `HuggingFace` pour charger le modèle `FLAN-T5` de Google.


In [4]:
# import packages
from langchain import PromptTemplate, HuggingFaceHub, LLMChain


# Load the model from HuggingFace
flan_t5 = HuggingFaceHub(
    repo_id="google/flan-t5-large",
    model_kwargs={"temperature": 1e-10},
)

# Build prompt template for simple question-answering
template = """
Question: {question}
Answer: 
"""

# Create the LangChain prompt template
prompt = PromptTemplate(template=template, 
                        input_variables=["question"])

# Create the LLMChain instance (A simple chain)
llm_chain = LLMChain(
    prompt=prompt, # prompt template
    llm=flan_t5 # LLM
)

# Define the question
question = "Quelle est la capital de la  France?"

# Run the chain and print the result
print(llm_chain.invoke(question))

{'question': 'Quelle est la capital de la  France?', 'text': 'paris'}


Il est possible de poser plusieurs questions à la fois d'un seul coup.

In [5]:
# basic prompt
multi_template = """
Répondre à chacune des questions ci-dessous.

Questions:
{questions}

Answers:
"""

# prompt template
long_prompt = PromptTemplate(template=multi_template, input_variables=["questions"])

# Create the LLMChain instance
llm_chain = LLMChain(
    prompt=long_prompt,
    llm=flan_t5
)

# list all the questions
qs_str = (
    "Quelle est la Capital de la France?\n" +
    "Combien font 1 + 1?\n" +
    "Who was the 12th person on the moon?"
)

print(llm_chain.invoke(qs_str))

{'questions': 'Quelle est la Capital de la France?\nCombien font 1 + 1?\nWho was the 12th person on the moon?', 'text': 'Quelle est la Capitale de la France? Combien font 1 + 1? Who was'}


On peut voir les limitations du modèle FLAN-T5 qui peut-être utilisé pour des questions simples mais qui ne peut pas répondre à des questions plus complexes.

C'est la raison pour laquelle nous allons utiliser un modèle plus puissant comme GPT-3 de OpenAI.

## Utilisation de Open AI

Commençons par montrer ce que cela donne lorsque l'on utilise les fonctions natives de OpenAI.

In [6]:
#Note: The openai-python library support for Azure OpenAI is in preview.
      #Note: This code sample requires OpenAI Python library version 1.0.0 or higher.
from openai import AzureOpenAI
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv())


llm = AzureOpenAI(
  azure_endpoint = os.getenv('AZURE_API_ENDPOINT'), 
  api_key=os.getenv("AZURE_OPENAI_KEY"),  
  api_version=os.getenv('OPENAI_API_VERSION')
)

# define the message
message_text = [{
    "role":"system",
    "content":"You are an AI assistant that helps people find information."}]

# get the completion
completion = llm.chat.completions.create(
  model="gpt35turbo", # model = "deployment_name"
  messages = message_text,
  temperature=0.0,
  max_tokens=800,
  top_p=0.95,
  frequency_penalty=0,
  presence_penalty=0,
  stop=None
)

# print the result
print(completion.choices[0].message.content)

Hello! I am an AI assistant designed to help you find information. What can I help you with today?


In [7]:
# give more elements
response = llm.chat.completions.create(
    model="gpt35turbo", # model = "deployment_name".
    messages=[
        {"role": "system", "content": "You are a helpful assistant."},
        {"role": "user", "content": "Does Azure OpenAI support customer managed keys?"},
        {"role": "assistant", "content": "Yes, customer managed keys are supported by Azure OpenAI."},
        {"role": "user", "content": "Do other Azure AI services support this too?"}
    ]
)

print(response.choices[0].message.content)

Yes, many other Azure AI services support customer managed keys, including Azure Cognitive Services, Azure Machine Learning, and Azure Databricks. Customer managed keys provide enhanced security and control over data encryption and are a recommended practice for many Azure services.


Maintenant avec LangChain.

In [3]:
import os
from langchain import PromptTemplate, LLMChain
#from langchain.chat_models import AzureChatOpenAI # OLD
from langchain_openai import AzureChatOpenAI # Chat model abstraction class


llm = AzureChatOpenAI(
    openai_api_version="2023-09-01-preview",
    azure_endpoint=os.getenv('AZURE_API_ENDPOINT'),
    api_key=os.getenv('AZURE_OPENAI_KEY'),
    azure_deployment=os.getenv('OPENAI_DEPLOYMENT_NAME'),
    model_name=os.getenv('OPENAI_MODEL_NAME'),
    model_version=os.getenv('OPENAI_API_VERSION')  # Add the desired version identifier
)


# Define the prompt template
multi_template = """
Répondre à chacune des questions ci-dessous.

Questions:
{questions}

Answers:
"""

# Create the prompt template
long_prompt = PromptTemplate(template=multi_template, input_variables=["questions"])

# Create the LLMChain instance
llm_chain = LLMChain(
    prompt=long_prompt, # prompt template
    llm=llm # Azure openai LLM
)

# List all the questions
qs_str = (
    "Quelle est la Capitale de la France?\n" +
    "Combien font 1 + 1?\n" +
    "Who was the 12th person on the moon?"
)

print(llm_chain.run(qs_str))


c:\Users\sebas\Documents\projet\data-atelier-intro-langchain\.venv\Lib\site-packages\langchain_core\_api\deprecation.py:117: LangChainDeprecationWarning: The function `run` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(


La Capitale de la France est Paris.
1 + 1 égalent 2.
Le 12ème homme sur la lune était Eugene Cernan.


On peut voir la puissance d'un modèle comme GPT-3 de OpenAI.

## Récap:

Les chaînes constituent le cœur de LangChain. Il s'agit simplement d'une chaîne de composants, exécutés dans un ordre particulier.

La plus simple de ces chaînes est la chaîne `LLMChain`. Elle fonctionne en prenant l'entrée d'un utilisateur, en la passant au premier élément de la chaîne - un PromptTemplate - pour formater l'entrée en une invite particulière. L'invite formatée est ensuite transmise à l'élément suivant (et final) de la chaîne, un LLM.

Les chaînes sont divisées en trois types : Les chaînes utilitaires, les chaînes génériques et les chaînes de combinaison de documents. Dans cette édition, nous nous concentrerons sur les deux premières, la troisième étant trop spécifique (elle sera abordée en temps voulu).

- Chaînes utilitaires : chaînes généralement utilisées pour extraire une réponse spécifique d'un formulaire de demande de renseignements, dans un but très précis, et prêtes à l'emploi.
- Chaînes génériques : chaînes utilisées comme blocs de construction pour d'autres chaînes, mais qui ne peuvent pas être utilisées seules.

In [7]:
# Utility chains example
from langchain.chains import LLMChain, LLMMathChain, TransformChain, SequentialChain
from langchain.callbacks import get_openai_callback

def count_tokens(chain, query):
    """
    Count the number of tokens used by the chain.
    """
    with get_openai_callback() as cb:
        result = chain.run(query)
        print(f'Spent a total of {cb.total_tokens} tokens')

    return result

llm_math = LLMMathChain.from_llm(llm=llm, 
                                 verbose=True) # to see what is going on

# call the llm whil 
count_tokens(llm_math, "What is the square root of 16?")




> Entering new LLMMathChain chain...
What is the square root of 16?```text
16**(0.5)
```
...numexpr.evaluate("16**(0.5)")...

Answer: 4.0
> Finished chain.
Spent a total of 231 tokens


'Answer: 4.0'

In [8]:
# see inside the prompt
print(llm_math.prompt.template)

Translate a math problem into a expression that can be executed using Python's numexpr library. Use the output of running this code to answer the question.

Question: ${{Question with math problem.}}
```text
${{single line mathematical expression that solves the problem}}
```
...numexpr.evaluate(text)...
```output
${{Output of running the code}}
```
Answer: ${{Answer}}

Begin.

Question: What is 37593 * 67?
```text
37593 * 67
```
...numexpr.evaluate("37593 * 67")...
```output
2518731
```
Answer: 2518731

Question: 37593^(1/5)
```text
37593**(1/5)
```
...numexpr.evaluate("37593**(1/5)")...
```output
8.222831614237718
```
Answer: 8.222831614237718

Question: {question}

